# Load Ni-Mo data

In [1]:
from pymatgen import Structure
from monty.serialization import loadfn

data = loadfn('data.json')
train_structures = [d['structure'] for d in data]
train_energies = [d['outputs']['energy'] for d in data]
train_forces = [d['outputs']['forces'] for d in data]

vasp_stress_order = ['xx', 'yy', 'zz', 'xy', 'yz', 'xz']
snap_stress_order = ['xx', 'yy', 'zz', 'yz', 'xz', 'xy']
train_stresses = []
for d in data:
    virial_stress = d['outputs']['stress']
    train_stresses.append([virial_stress[vasp_stress_order.index(n)] * 0.1 for n in snap_stress_order]) # convert kbar to GPa

# Setup the initial weights for training (If not, the weights for energy, force, and stress will be both equal to 1)

In [2]:
import numpy as np
from maml.utils import pool_from, convert_docs

train_pool = pool_from(train_structures, train_energies, train_forces, train_stresses)
_, df = convert_docs(train_pool, include_stress=True)

weights = np.ones(len(df['dtype']), )

# set the weights for energy equal to 100
weights[df['dtype'] == 'energy'] = 100
weights[df['dtype'] == 'force'] = 1
weights[df['dtype'] == 'stress'] = 0.01

# Set up the SNAP and train

In [3]:
from maml.base import SKLModel
from maml.describers import BispectrumCoefficients
from sklearn.linear_model import LinearRegression
from maml.apps.pes import SNAPotential

element_profile = {'Mo': {'r': 5.0, 'w': 1}, 'Ni': {'r': 5.0, 'w': 1}}
describer = BispectrumCoefficients(rcutfac=0.5, twojmax=6, element_profile=element_profile, 
                                   quadratic=True, pot_fit=True, include_stress=True)
model = SKLModel(describer=describer, model=LinearRegression())
qsnap = SNAPotential(model=model)
qsnap.train(train_structures, train_energies, train_forces, train_stresses, include_stress=True, sample_weight=weights)

INFO:maml.apps.pes._lammps:Setting Lammps executable to lmp_serial
INFO:maml.utils._lammps:Structure index 0 is rotated.
INFO:maml.utils._lammps:Structure index 1 is rotated.
INFO:maml.utils._lammps:Structure index 2 is rotated.
INFO:maml.utils._lammps:Structure index 3 is rotated.
INFO:maml.utils._lammps:Structure index 4 is rotated.
INFO:maml.utils._lammps:Structure index 5 is rotated.
INFO:maml.utils._lammps:Structure index 6 is rotated.
INFO:maml.utils._lammps:Structure index 7 is rotated.
INFO:maml.utils._lammps:Structure index 8 is rotated.
INFO:maml.utils._lammps:Structure index 9 is rotated.


# Predict the energies, forces, stresses of training data

In [4]:
df_orig, df_predict = qsnap.evaluate(test_structures=train_structures, 
                                     test_energies=train_energies,
                                     test_forces=train_forces, 
                                     test_stresses=train_stresses,
                                     include_stress=True)

INFO:maml.utils._lammps:Structure index 0 is rotated.
INFO:maml.utils._lammps:Structure index 1 is rotated.
INFO:maml.utils._lammps:Structure index 2 is rotated.
INFO:maml.utils._lammps:Structure index 3 is rotated.
INFO:maml.utils._lammps:Structure index 4 is rotated.
INFO:maml.utils._lammps:Structure index 5 is rotated.
INFO:maml.utils._lammps:Structure index 6 is rotated.
INFO:maml.utils._lammps:Structure index 7 is rotated.
INFO:maml.utils._lammps:Structure index 8 is rotated.
INFO:maml.utils._lammps:Structure index 9 is rotated.


# Lattice constants, Elastic constant
### Large error due to limited training data -- 10 structures

In [ ]:
from pymatgen.core import Lattice

Ni = Structure.from_spacegroup(sg='Fm-3m', species=['Ni'], lattice=Lattice.cubic(3.51), coords=[[0, 0, 0]])
Mo = Structure.from_spacegroup(sg='Im-3m', species=['Mo'], lattice=Lattice.cubic(3.17), coords=[[0, 0, 0]])

In [5]:
from maml.apps.pes import ElasticConstant

Ni_ec_calculator = ElasticConstant(ff_settings=qsnap)
Ni_C11, Ni_C12, Ni_C44, _ = Ni_ec_calculator.calculate([Ni])[0]
print('Ni', ' C11: ', Ni_C11, 'C12: ', Ni_C12, 'C44: ', Ni_C44)

INFO:maml.apps.pes._lammps:Setting Lammps executable to lmp_serial


Ni  C11:  5736938392.20117 C12:  -569243687.375 C44:  -1306336862.75


In [6]:
Mo_ec_calculator = ElasticConstant(ff_settings=qsnap)
Mo_C11, Mo_C12, Mo_C44, _ = Mo_ec_calculator.calculate([Mo])[0]
print('Mo', ' C11: ', Mo_C11, 'C12: ', Mo_C12, 'C44: ', Mo_C44)

INFO:maml.apps.pes._lammps:Setting Lammps executable to lmp_serial


Mo  C11:  -10043717.1291321 C12:  631379.849702305 C44:  -776527.731726994
